# General Information 
This is the report for the [mass-spring-damper](https://github.com/citros-garden/mass_spring_dumper) project.
In the project we created a mass-spring-damper system, controlled by a PID controller.
Each element in the simulation (the dynamics system and the controller) implemented in a ROS 2 node.

## System dynamics
The system's equations of motion:
$$m\ddot x =  kf(t) -c\dot x -kx$$
and after laplace transformation (with zero I.C) we get a second order system:
$${X \over F} = {\omega_n^2 \over s^2 +2\omega_n\zeta s + \omega_n^2} $$
where the natural frequency $\omega_n = \sqrt{k \over m}$
You can choose the system's parameters `m`, `k` and `c` and choose the initial condition `x0`, `v0` and `a0` as parameters.

## The controller
the controller is a simple PID controller with the following form:
$$f(t) = {k_pe(t) + k_i\int{e(t)dt}} + k_d {d\over dt}(e(t))$$
you can tune the controller gains, $k_p$, $k_i$, and $k_d$ as they also define as ROS 2 parameters.




# The Scenario 
We wish to validate the robustness of our PID gain tunning for normal distributed mass.\
Running a batch of simulation with random value of the mass picked from normal distribution as:
$$ m = N(\mu, \sigma) $$

Where:

$$ \mu = 1.0 $$

$$ \sigma = 0.3 $$

First, lets import the relevant packages for the analysis:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from citros import CitrosDB
from prettytable import PrettyTable, ALL

# The Requierments 



1. The mass overshoot should not exceed `30%`

2. The mass should settled to `10%` of the steady state in under `3.0 [sec]`



We will define those requierments and some function that will help us detect if we not crossing any requierments.

In [ ]:
requirements = {
    "max_overshoot": 0.3,
    "settling_value": 0.1,
    "setteling_time": 3.0
}

def _find_nearest(array: list, value: float) -> int:
    '''
        return the index of the nearest element to value in a given array.
    '''
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def _test_overshoot(data: list) -> int:
    '''
        test if the overshoot exceed the requirments. return 0 if not.
    '''
    overshoot = max(data)
    if overshoot > requirements["max_overshoot"]:
        return 1
    else:
        return 0

def _test_settling_time(data: list, time: list) -> int:
    '''
        test if the settling time is in the requirements. return 0 if it is.
    '''
    nn = _find_nearest(time, requirements["setteling_time"])
    if requirements["settling_value"] - abs(data[nn]) < 0:
        return 1
    else:
        return 0

def detect_failed_test(data: list, time: list) -> int:
    '''
        check if the simulation pass the requirements.
    '''
    return _test_overshoot(data) + _test_settling_time(data, time)

We can define a new function, which will function as a wrapper for `citros.plot_graph` and will plot graph for each batch:

In [ ]:


def plot_batch(simulation, batch):
    '''
        import data from CITROS database, and plot the mass position with the requirements lines.
    '''

    citros = CitrosDB(simulation =  simulation, batch = batch)
    print(citros.info())

    # get the mass position from CITROS
    controlled_system = citros.topic('/dynamics/position').sid().data('data.data')

    # convert rid to time based on dt
    controlled_system['rid'] = controlled_system['rid'].apply(lambda x: x * 0.003)

    # get the mass parameter for each simulation
    mass_raw = citros.topic('/config').sid().data('data.dynamics.ros__parameters.m')
    mass_legends = [f"m = {x:.4} [kg]" for x in mass_raw['data.dynamics.ros__parameters.m'] if "nan" not in str(x)]
    mass = [x for x in mass_raw['data.dynamics.ros__parameters.m'] if "nan" not in str(x)]

    # get the PID gains for each simulation
    kp_raw = citros.topic('/config').sid().data('data.pid.ros__parameters.kp')
    kp = [x for x in kp_raw['data.pid.ros__parameters.kp'] if "nan" not in str(x)][0]

    ki_raw = citros.topic('/config').sid().data('data.pid.ros__parameters.ki')
    ki = [x for x in ki_raw['data.pid.ros__parameters.ki'] if "nan" not in str(x)][0]

    kd_raw = citros.topic('/config').sid().data('data.pid.ros__parameters.kd')
    kd = [x for x in kd_raw['data.pid.ros__parameters.kd'] if "nan" not in str(x)][0]

    # creating graph for all the simulations runs.
    fig, ax = citros.plot_graph(controlled_system, 'rid', 'data.data')

    # define max rid (time) for plotting
    max_rid = 0

    # define counters
    passed_test = 0
    failed_test = 0
    invalid_test = 0
    passed_masses = []

    # now iterate over each line in the graph. if the line passes the tests (from requirements) set the line color to green and
    # update passed_test counter. if not, set the line color to red and update invalid_test. if the mass is negative, continue and 
    # update invalid_test.

    for counter, line in enumerate(ax.get_lines()):

        if mass[counter] < 0:
            invalid_test += 1
            line.set_color('k')
            continue

        simulation_data = line.get_data()[1]
        simulation_time = line.get_data()[0]
        max_rid = max_rid if max_rid > max(simulation_time) else max(simulation_time)

        if not detect_failed_test(simulation_data, simulation_time):
            line.set_color('g')
            passed_masses.append(mass[counter])
            passed_test += 1

        else:
            line.set_color('r')
            failed_test += 1

    # adding the requirements to the graph
    ax.axhline(requirements["max_overshoot"], xmin=0, xmax=max_rid, c='r', ls='--')
    ax.text(max_rid - max_rid / 3, requirements["max_overshoot"] + requirements["max_overshoot"] / 4, "Maximum Overshoot")

    ax.axhline(requirements["settling_value"], xmin=0, xmax=max_rid, c='b', ls='--')
    ax.axhline(-requirements["settling_value"], xmin=0, xmax=max_rid, c='b', ls='--')
    ax.text(max_rid - max_rid / 3, requirements["settling_value"] + requirements["settling_value"] / 2, "Settling Value")

    ax.axvline(requirements["setteling_time"], ymin=-1.5, ymax=1.0, c='b', ls='--')
    ax.text(requirements["setteling_time"] + requirements["setteling_time"] / 8, 0.75, "Settling Time")

    ax.get_legend().remove()
    ax.grid(False)

    # setting a title with information about the batch runs
    ax.set_title(f"Simulation = {simulation}, Batch = {batch}\nPassed: {passed_test}\nFailed: {failed_test}\nInvalid: {invalid_test}\
    \nMaximum mass that passed the test: {max(passed_masses):.4} [kg]\
    \nMinimum mass that passed the test: {min(passed_masses):.4} [kg]\
    \nPID Gains: Kp = {kp}, Ki = {ki}, Kd = {kd}")

    ax.set_ylim([-1.5, 1])
    ax.set_xlabel("Time [sec]")
    ax.set_ylabel("Position [m]")

Now we can import our tests ID from CITROS and plot them to get the results:

In [ ]:

plot_batch("simulation_dynamics_controller","dynamics_controller")